# Prerequisites (clone tensorflow_models)

In [ ]:
!git clone -q https://github.com/rok/models
!cd models && git checkout mobilenet_v2_tpu_with_dist_strat_and_keras

!pip install -q tf-nightly-2.0-preview

In [ ]:
import sys
sys.path.append('/content/models/research/')

# Checkpoint based inference

In [ ]:
import os
import tensorflow as tf
from object_detection.models.keras_models import mobilenet_v2
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
file = tf.keras.utils.get_file(
    "panda.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG")
img = tf.keras.preprocessing.image.load_img(file, target_size=[224, 224])
plt.imshow(img)
plt.axis('off')

x = tf.keras.applications.mobilenet_v2.preprocess_input(np.array(img)[tf.newaxis,...])

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2()
result = pretrained_model.predict(x)
tf.keras.applications.mobilenet_v2.decode_predictions(result)

# Frozen inference

In [ ]:
tf.saved_model.save(pretrained_model, "/tmp/mobilenet_v2/1/")

In [ ]:
loaded = tf.saved_model.load("/tmp/mobilenet_v2/1/")
infer = loaded.signatures['serving_default']
labeling = infer(tf.constant(x))['Logits']

tf.keras.applications.mobilenet_v2.decode_predictions(labeling.numpy())

# TPU inference

In [ ]:
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime.')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  tf.config.experimental_connect_to_host(tpu_address)


resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
tf.tpu.experimental.initialize_tpu_system(resolver)
tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
# TPU models are not available in TF2 yet
# model = tf.keras.applications.MobileNetV2()
# tpu_model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=tpu_strategy)